In [ ]:
sequence = 5
LR = 0.001
epoch_num = 30
feature_num = 108

In [ ]:
import time
import os
from Research.backtest.bt import BTDaily
import matplotlib.pyplot as plt
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp
import sys
sys.path.append("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/packages/")
from AlphaNet.Data import DataLoader , Model_Loader
from AlphaNet.Models import AlphaNet_LSTM_V1
model_path = "/home/wuwenjun/Alpha_Factor/" + Alpha_Name + "/" + "%s_%s" %(start_date,end_date) + "/"
if os.path.exists(model_path) == False:
    os.makedirs(model_path)
    print(model_path)
data_path = "/home/ShareFolder/feature_platform/ti0/wuwenjun/#Data_lib/"



# Train Loader

trainloader = DataLoader()
trainloader.load_data_from_file(alpha_name = Alpha_Name,end_date = start_date,data_path=data_path)
train_loader = trainloader.to_torch_DataLoader(sequence = sequence,shuffle=True)

# Model Loader
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
loss_function = nn.MSELoss()
model = AlphaNet_LSTM_V1(feature_num, 64)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
model_loader = Model_Loader(model = model,device=device)
print(model_loader.model)

# Training
model = model_loader.fit_transform(train_loader,optimizer,loss_function,epoch_num,save_path = model_path)

optimizer = optim.Adam(model.parameters(), lr=LR/10)
model = model_loader.transform(optimizer = optimizer, epoch_num = epoch_num, method="best")

optimizer = optim.Adam(model.parameters(), lr=LR/100)
model = model_loader.transform(optimizer = optimizer, epoch_num = epoch_num, method="best")

# Test
testloader = DataLoader()
testloader.load_data_from_file(alpha_name = Alpha_Name,start_date = start_date,end_date = end_date)
test_loader = testloader.to_torch_DataLoader(sequence = 3,shuffle=False)
pred = model_loader.pred(test_loader)

# convert to standard daily and back test
final = pd.concat([testloader.target.reset_index(),pred],axis=1)
final.rename(columns={0:Alpha_Name,'ticker': 'symbol'},inplace=True)
final.to_parquet(model_path + "result.par")
convert_to_standard_daily_feature_csv(Alpha_name, df.drop("target",axis=1), output_path = r'/home/wuwenjun/factor_lib/ti0/wuwenjun')